In [16]:
from urllib import request 
from re import split, sub
import tiktoken 
import torch 
from torch.utils.data import Dataset, DataLoader 

#### uploade text

#### SOURCE: [Raschka, Sebastian. Build a Large Language Model (From Scratch). Simon and Schuster, 2024.](https://www.simonandschuster.com/books/Build-a-Large-Language-Model-From-Scratch/Sebastian-Raschka/9781394200867)

In [17]:
url = ("https://raw.githubusercontent.com/rasbt/"
"LLMs-from-scratch/main/ch02/01_main-chapter-code/"
"the-verdict.txt")
file_path = "the-verdict.txt"
request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x1da319f1c70>)

In [18]:
#book = None
with open("the-verdict.txt", "r", encoding="utf-8") as file:
    text = file.read()
    book = file.read()
    print(f"number of characters: {len(text)}")
    print(text[:300])
    
    

number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would ha


In [19]:
f = open("the-verdict.txt", "r", encoding="utf-8")
book = f.read()
result = split(r'([,.:;?_!"()\']|--|\s)', book)
print(book[:100])
print(result[:100])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g
['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that', ',', '', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory', ',', '', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting', ',', '', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow', ',', '', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ']


In [20]:
result = [item.strip() for item in result if item.strip()]

In [21]:
print(result[:100])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter', '--']


In [22]:
print(len(result))

4690


In [23]:
all_words = sorted(set(result))
all_words.extend(["<|endoftext|>", "<|unk|>"])
print(len(all_words))

1132


In [24]:
vocab = {token: integer for integer, token in enumerate(all_words)}

In [25]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >=20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [26]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)
    #if i >=20:
       # break

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [27]:
class Tokenizer:
    # Simple implementation of a tokenizer class
    def __init__(self, vocab):
        
        self.str_to_int = vocab
        self.int_to_str = {i:token for token,i in vocab.items()}

    def encoder(self, text):
        
        preprocessed = split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[token] for token in preprocessed]
        
        return ids

    def decoder(self, ids):
        
        text = " ".join(self.int_to_str[i] for i in ids)
        text = sub(r'\s+([,.?!"()\'])', r'\1', text) # remove spaces before the specified punctuation.

        return text

In [28]:
tokenizer = Tokenizer(vocab)
text = "It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."
ids = tokenizer.encoder(text)
text = tokenizer.decoder(ids)

In [29]:
print(ids)
print(text)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 7, 38, 851, 1108, 754, 793, 7]
It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride.


In [30]:
ids_ = tokenizer.encoder(book)
text_ = tokenizer.decoder(ids_)

In [31]:
print(ids_[:20])
print(text_[:100])

[53, 44, 149, 1003, 57, 38, 818, 115, 256, 486, 6, 1002, 115, 500, 435, 392, 6, 908, 585, 1077]
I HAD always thought Jack Gisburn rather a cheap genius -- though a good fellow enough -- so it was 


In [32]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text3 = " <|endoftext|> ".join((text1, text2))
print(text3)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [33]:
tokenizer1 = Tokenizer(vocab)
print(tokenizer1.encoder(text3))
print(tokenizer.decoder(tokenizer1.encoder(text3)))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [34]:
tokenizer = tiktoken.get_encoding("gpt2") #  instantiate the BPE (byte pair encoding) tokenizer from tiktoken library.

In [35]:
text = (
"Hello, do you like tea? <|endoftext|> In the sunlit terraces"
"of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [36]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [37]:
for int_ in integers:
    print(f"{int_}: {tokenizer.decode([int_])}")

15496: Hello
11: ,
466:  do
345:  you
588:  like
8887:  tea
30: ?
220:  
50256: <|endoftext|>
554:  In
262:  the
4252:  sun
18250: lit
8812:  terr
2114: aces
1659: of
617:  some
34680: unknown
27271: Place
13: .


####  Data sampling with a sliding window

In [38]:
tokenizer = tiktoken.get_encoding("gpt2")
with open("the-verdict.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [39]:
enc_sample = enc_text[50:]

In [40]:
context_size = 4 # number of tokens include in the input
x = enc_sample[:context_size] # input tokens
y = enc_sample[1: context_size + 1] # target tokens

print(f"x: {x}")
print(f'y: {y}')

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [41]:
print(tokenizer.decode(x))
print(tokenizer.decode(y))

 and established himself in
 established himself in a


In [42]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "----->", desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [43]:
for i in range(1, context_size+1):
    context = tokenizer.decode(enc_sample[:i])
    desired = tokenizer.decode([enc_sample[i]])
    print(context, "----->", desired)

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


In [44]:
# data loader for loading and tokenizing input text
class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride): 
        
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        
        for i in range(0, len(token_ids) - max_length, stride):
            
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [45]:
def create_dataloader(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(text, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, #drop_last=True drops the last batch if it is shorter than the specified batch_size to prevent loss spikes during training.
        num_workers=num_workers
    )
    
    return dataloader

In [46]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
dataloader = create_dataloader(
    text=raw_text,
    batch_size=5,
    max_length=10,
    stride=2,
    shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026, 15632,   438],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438,  2016,   257],
        [10899,  2138,   257,  7026, 15632,   438,  2016,   257,   922,  5891]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257],
        [ 1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632,   438,  2016],
        [  271, 10899,  2138,   257,  7026, 15632,   438,  2016,   257,   922],
        [ 2138,   257,  7026, 15632,   438,  2016,   257,   922,  5891,  1576]])]


In [47]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[  257,  7026, 15632,   438,  2016,   257,   922,  5891,  1576,   438],
        [15632,   438,  2016,   257,   922,  5891,  1576,   438,   568,   340],
        [ 2016,   257,   922,  5891,  1576,   438,   568,   340,   373,   645],
        [  922,  5891,  1576,   438,   568,   340,   373,   645,  1049,  5975],
        [ 1576,   438,   568,   340,   373,   645,  1049,  5975,   284,   502]]), tensor([[ 7026, 15632,   438,  2016,   257,   922,  5891,  1576,   438,   568],
        [  438,  2016,   257,   922,  5891,  1576,   438,   568,   340,   373],
        [  257,   922,  5891,  1576,   438,   568,   340,   373,   645,  1049],
        [ 5891,  1576,   438,   568,   340,   373,   645,  1049,  5975,   284],
        [  438,   568,   340,   373,   645,  1049,  5975,   284,   502,   284]])]


### Embedding vectors

In [48]:
torch.manual_seed(123)
vocab_size = 12
output_dim = 6
embedding_layer = torch.nn.Embedding(
    num_embeddings=vocab_size,
    embedding_dim=output_dim
)
print(embedding_layer.weight)
print(embedding_layer.weight.shape)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603],
        [-0.2196, -0.3792,  0.7671, -1.1925,  0.6984, -1.4097],
        [ 0.1794,  1.8951,  0.4954,  0.2692, -0.0770, -1.0205],
        [-0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010],
        [ 0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255],
        [-0.6315, -2.8400, -1.3250,  0.1784, -2.1338,  1.0524],
        [-0.3885, -0.9343, -0.4991, -1.0867,  0.8805,  1.5542],
        [ 0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850],
        [ 0.8768,  1.6221, -1.4779,  1.1331, -1.2203,  1.3139],
        [ 1.0533,  0.1388, -0.0635, -0.4486, -1.2785, -1.1440],
        [ 0.2436, -0.0567,  0.3784,  1.6863, -0.5581, -0.0517],
        [-0.9083,  0.3507,  1.5434,  0.1406,  1.0617, -0.9929]],
       requires_grad=True)
torch.Size([12, 6])


In [49]:
input_ids = torch.tensor([1, 2, 3, 4]) # token ids for 4 vcabs
print(embedding_layer(input_ids)) # applying embedding weights to input vocabs

tensor([[-0.2196, -0.3792,  0.7671, -1.1925,  0.6984, -1.4097],
        [ 0.1794,  1.8951,  0.4954,  0.2692, -0.0770, -1.0205],
        [-0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010],
        [ 0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255]],
       grad_fn=<EmbeddingBackward0>)


In [50]:
vocab_size = 50257 # vocabolary size of BEP tokenizer
output_dim = 256

token_embedding_layer = torch.nn.Embedding(
    num_embeddings=vocab_size,
    embedding_dim=output_dim
)

In [51]:
print(token_embedding_layer.weight.shape)

torch.Size([50257, 256])


#### Using the previous token_embedding_layer, if we sample data from the data loader,
#### we embed each token in each batch into a 256-dimensional vector. If we have a batch
#### size of 8 with four tokens each, the result will be an 8 × 4 × 256 tensor.

In [66]:
max_length = 4
dataloader = create_dataloader(
    text=raw_text,# the verdict roman
    batch_size=8,
    max_length=max_length,
    stride=max_length,
    shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter) # first batch : 1. inputs, 2. targets
print(first_batch[0])
print(first_batch[0].shape)  # the token ID tensor is 8 × 4 dimensional, meaning that the data batch consists of eight text samples with four tokens each.

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
torch.Size([8, 4])


In [67]:
token_embeddings = token_embedding_layer(first_batch[0]) # embedding first batich inputs (token IDs) into 256-dimensional vectors
print(token_embeddings.shape)  # each token ID is now embedded as a 256-dimensional vector.

torch.Size([8, 4, 256])


In [68]:
# For a GPT model’s absolute embedding approach, we just need to create another
# embedding layer that has the same embedding dimension as the token_embedding_layer:

context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [69]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
